### 数据表,数据录入

    drop table if exists Advertising;
    create table Advertising(
        id bigserial,
        tv float8,
        radio float8,
        newpaper float8,
        sales float8
    );

In [ ]:
#录入数据
import pandas
df=pandas.read_csv('data/Advertising.csv')
sqls=''
for i in range(len(df)):
    d=df.iloc[i]
    sql="insert into Advertising(tv,radio,newpaper,sales) values('%.2f','%.2f','%.2f','%.2f');"%(d['TV'],d['Radio'],d['Newspaper'],d['Sales'])
    sqls=sqls+sql+'\n'
with open('data/Advertising.sql','w') as fs:
    fs.write(sqls)
#执行脚本命令
#\i ..../Advertising.sql

### $Y=f(X_1,X_2,X_3...X_n)$
$X_1,X_2,X_3...X_n是因变量,Y是自编量$

 输入x,输出Y

## A.使用线性模型 训练数据
    目的:$sales=F(tv,radio,newpaper)$
$Y=\beta_1X_1+\beta_2X_2+\beta_3X_3...+\beta_nX_n$

    select madlib.linregr_train(
        'Advertising',   --源表 
        'Advertising_param',--输出表
        'sales', --因变量 Y
        'array[1,tv,radio,newpaper]' --自编量变量X
        );

      select coef,r2 from advertising_param; --#查看参数,R2

### #sale=2.93+0.04576*tv+0.18853*radio+-0.001*newpaper,R2=0.89,说明线性模型效果不是很好
<img src='img/linear_result.png'>



##  使用训练好的程序预测

    select t1.tv,        --参与计算的属性tv
           t1.radio,     --参与计算的属性radio
           t1.newpaper , --参与计算的属性newpaper
           madlib.linregr_predict(m.coef,array[1,tv,radio,newpaper]) as predict, --使用训练好的参数预测sale
           sales --真正的sales,对比使用
           from 
                Advertising t1,  --需要预测的数据,
                Advertising_param m; --参数表

<img src='img/l2.png'/>

## B.使用广义线性模型(GLM)训练数据,做数据拟合
[API]http://madlib.apache.org/docs/latest/group__grp__glm.html

    select madlib.glm('Advertising',  --源表
                      'Advertising_glm_result', --目标表
                      'sales',  --因变量
                      'array[1,tv,radio,newpaper]', --自编量
                      'family=gamma,link=log' --不同于linear regress,这里Y假设服从gamma分布,使用log作为链接函数
                     );
$log(E[Y|X])=X^T\beta=\beta_1X_1+\beta_2X_2+\beta_3X_3...+\beta_nX_n$

##  使用训练好的GLM预测

    select t1.tv,        --参与计算的属性tv
           t1.radio,     --参与计算的属性radio
           t1.newpaper , --参与计算的属性newpaper
           madlib.glm_predict(m.coef,array[1,tv,radio,newpaper],'log') as predict, --使用训练好的参数预测sale
           sales --真正的sales,对比使用
           from 
                Advertising t1,  --需要预测的数据,
                Advertising_glm_result m; --参数表

<img src='img/l3.png'>

## C.弹性回归
https://madlib.apache.org/docs/latest/group__grp__elasticnet.html#additional_example

$l(\beta)=\sum_{1}^{N}(X_i^T\beta-y_i)^2+\lambda(\alpha*\|\beta\|_1+(1-\alpha)+\|\beta\|_2^2)$

1.上述的线性回归+正则化

2.$\alpha=1$是lasso回归,$\alpha=0$是岭回归

## 训练函数
    drop table if exists Advertising_elastic,Advertising_elastic_summary ;
    select madlib.elastic_net_train('Advertising',--输入数据表
                       'Advertising_elastic',--输出模型表
                       'sales',--因变量
                       'array[tv,radio,newpaper]',--自变量
                       'gaussian',--回归问题是gaussian,分类问题是binomial
                       0.5,--上面的alpha
                       0.1,--上面的lambda
                       true--列是否标准化,缩放到[0,1]
                     );
      # select * from Advertising_elastic;--查看模型表
<img src='img/elastic1.png' />

## 预测数据

    select madlib.elastic_net_gaussian_predict(
                        m.coef_nonzero, --模型的系数
                        m.intercept,--模型的截距
                        array[tv,radio,newpaper]--自变量名称
                       )
        from Advertising,     --输入数据表
        Advertising_elastic m --模型表;
<img src='img/elastic2.png' />